In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re

driver = webdriver.Chrome('./chromedriver')

# Save to JSON and load

In [2]:
import json

def save_list_to_json(data,filename):
    json_string = json.dumps(data)
    with open(filename+'.json', 'w') as f:
        json.dump(data, f)



def load_json(filename):
    with open(filename+'.json') as f:
        data = json.load(f)
        return data 

# Get events id

In [3]:

events_source=['https://www.hltv.org/stats/matches?matchType=Majors']
i=50
while i<700:
    lk='https://www.hltv.org/stats/matches?matchType=Majors&offset='+str(i)
    i+=50
    events_source.append(lk)

In [4]:
events_source

['https://www.hltv.org/stats/matches?matchType=Majors',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=50',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=100',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=150',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=200',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=250',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=300',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=350',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=400',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=450',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=500',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=550',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=600',
 'https://www.hltv.org/stats/matches?matchType=Majors&offset=650']

In [ ]:
#add date,event names
def get_matches_pages(source):
    event_lists=[]
    html_list=[]
    for sc in source:
            driver.get(sc)
            html = driver.page_source
            soup = BeautifulSoup(html)
            soup=soup.findAll('tr')
            html_list+=[html]
            for tr in soup:
                td=tr.findAll('td', {"class": "date-col"})
                for tag in td:
                    lk=tag.select_one("a[href*=matches]")['href']
                    if len(lk)>4 and not lk in event_lists:
                        event_lists.append(lk)
   
    return event_lists,html
        

### Get match pages links and html

In [ ]:
matches_list,html=get_matches_pages(events_source)



In [ ]:
save_list_to_json(matches_list,'scraping/major_matches_links')
save_list_to_json(html,'scraping/major_matches_html')


In [6]:
matches_list=load_json('scraping/major_matches_links')

# Get matches links

In [7]:
def divide_chunks(l, n):
      
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]
  


  
sublist = list(divide_chunks(matches_list, 20))

In [8]:
len(sublist)

35

In [17]:
match_url='https://www.hltv.org'

    
def get_matches_html(urls):
    ##### get inner demo page links first
    links=[]
    html1={}
    html2={}
    for url in urls:
            driver.get('https://www.hltv.org'+url)
            html = driver.page_source
            soup = BeautifulSoup(html)
            divs=soup.findAll('div', {"class": "match-info-box-con"})
            
            for div in divs:
                
                lk=div.findAll('a', {"class": "match-page-link"})
              
                for l in lk:
                    a=l['href']
                    if  len(a)>4 and not a in links:
                        links.append(a)
                        html1[a]=html
    ##### get demo links
    demos=[]
    for url in links:
            driver.get('https://www.hltv.org'+url)
            html = driver.page_source
            soup = BeautifulSoup(html)
            
            #https://www.hltv.org/matches/2290747/ldlc-vs-3dmax-ems-one-katowice-2014
            try:
                demo_link=soup.select_one("a[href*=download]")['href']
                if  "demo" in demo_link and not demo_link in demos:
                            demos.append(demo_link)

                            html2[url]=html
            except:
                print("no demo")
            else:
                pass
    
    return demos,html1,html2 

In [18]:
end=[]
#Change to avoid ip ban
for i in range(len(sublist)):
    end+=sublist[i]

### Save matches html page content

In [19]:
demos,html1,html2=get_matches_html(end)
print('done')
#load current data
a=load_json('scraping/major_demo_links')
b=load_json('scraping/major_stat_html')
c=load_json('scraping/major_matchpage_html')
#add new data
demos+=a
html1.update(b)
html2.update(c)
#save updated data
save_list_to_json(list(set(demos)),'scraping/major_demo_links')
save_list_to_json(html1,'scraping/major_stat_html')
save_list_to_json(html2,'scraping/major_matchpage_html')

print(len(list(set(demos))))


no demo
done
446


# Download demos

In [4]:
demos=load_json('scraping/major_demo_links')


In [5]:
downloaded_demo={}


In [6]:
import urllib,shutil
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
hltv='https://www.hltv.org'


for i in range(431
               ,446):
    page=urllib.request.Request(hltv+demos[i],headers=hdr) 
    # Download the file from `url` and save it locally under `file_name`:
    with urllib.request.urlopen(page) as response, open(str(i)+'.rar', 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
        downloaded_demo[str(i)]=True
        a=load_json('scraping/downloaded_demos')
        downloaded_demo.update(a)
        save_list_to_json(downloaded_demo,'scraping/downloaded_demos')
        


# Get Demo links

In [ ]:
for i in matches_links:
    print(i['match_link'])

In [4]:
matches_info=load_json('scraping/matches_links')

In [ ]:
#save htlm code first

def get_matches_html(urls):
    ##### get inner demo page links first
    links=[]
    html1=[]
    html2=[]
    for url in urls:
            driver.get('https://www.hltv.org'+url['match_link'])
            html = driver.page_source
            soup = BeautifulSoup(html)
            divs=soup.findAll('div', {"class": "match-info-box-con"})
            
            for div in divs:
                
                lk=div.findAll('a', {"class": "match-page-link"})
              
                for l in lk:
                    a=l['href']
                    if  len(a)>4 and not a in links:
                        links.append(a)
                        html1.append(html)
    ##### get demo links
    demos=[]
    for url in links:
            driver.get('https://www.hltv.org'+url)
            html = driver.page_source
            soup = BeautifulSoup(html)
            
            
            demo_link=soup.select_one("a[href*=download]")['href']
            if  "demo" in demo_link and not demo_link in demos:
                        demos.append(demo_link)
                        if   not html in html2:
                            html2.append(html)
    
    return demos,html1,html2


In [ ]:
demos,html1,html2=get_matches_html(matches_info)

In [ ]:
#add date, maps,teamsid,teams names, player id, player name, ratings,stats?,top player,html, bo score, round scores
def get_matches_links(urls):
    ##### get inner demo page links first
    links=[]
    
    for url in urls:
            driver.get('https://www.hltv.org'+url)
            html = driver.page_source
            soup = BeautifulSoup(html)
            divs=soup.findAll('div', {"class": "match-info-box-con"})
            
            for div in divs:
                
                lk=div.findAll('a', {"class": "match-page-link"})
              
                for l in lk:
                    a=l['href']
                    if  len(a)>4 and not a in links:
                        links.append(a)
    ##### get demo links
    demos=[]
    for url in links:
            driver.get('https://www.hltv.org'+url)
            html = driver.page_source
            soup = BeautifulSoup(html)
            
            
            demo_link=soup.select_one("a[href*=download]")['href']
            if  "demo" in demo_link and not demo_link in demos:
                        demos.append(demo_link)
    
    return demos



In [ ]:
demo_links=get_matches_links(my_events_links)

In [ ]:
save_list_to_json(demo_links,'scraping/demo_links')